In [10]:
import pandas as pd
from pathlib import Path
from sklearn.decomposition import LatentDirichletAllocation
from nltk.stem.porter import PorterStemmer
import re
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import xgboost as xgb
import dalex as dx
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

E:\anaconda\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [4]:
df = pd.read_json(Path.cwd() / "final_df" / "final_df.json")

In [5]:
df

,title,link,citations,text
0,Continual Reinforcement Learning with TELLA,http://arxiv.org/pdf/2208.04287v1,2,Workshop Track - 1st Conference on Lifelong Le...
1,An exact mapping between the Variational Renor...,http://arxiv.org/pdf/1410.3831v1,295,arXiv:1410.3831v1 [stat.ML] 14 Oct 2014An ex...
2,Learning Generative Models across Incomparable...,http://arxiv.org/pdf/1905.05461v2,69,Learning Generative Models across Incomparable...
3,On the Generalization Ability of Online Learni...,http://arxiv.org/pdf/1305.2505v1,74,On the Generalization Ability of Online Learni...
4,Geometric Understanding of Deep Learning,http://arxiv.org/pdf/1805.10451v2,110,Geometric Understanding of Deep Learning\nNa L...
...,...,...,...,...
1230,Musical Word Embedding: Bridging the Gap betwe...,http://arxiv.org/pdf/2008.01190v1,4,Musical Word Embedding: Bridging the Gap betwe...
1231,Metrics for Multi-Class Classification: an Ove...,http://arxiv.org/pdf/2008.05756v1,409,METRICS FOR MULTI -CLASS CLASSIFICATION :ANOVE...
1232,Intelligence plays dice: Stochasticity is esse...,http://arxiv.org/pdf/2008.07496v1,5,Intelligence plays dice: Stochasticity is esse...
1233,Can AutoML outperform humans? An evaluation on...,http://arxiv.org/pdf/2009.01564v2,24,Can AutoML outperform humans? An evaluation \n...


In [6]:
df['text'].isna().sum()

12

In [7]:
df.dropna(subset=['text'], inplace=True)

In [8]:
stemmer = PorterStemmer()
tokenizer = RegexpTokenizer(r'\w+')
def preprocess_text(text: str):
    return ' '.join([stemmer.stem(word) for word in tokenizer.tokenize(' '.join([re.sub(r'\s+', ' ', word) for word in text.split()]))])


In [9]:
df['text_pp'] = df['text'].apply(lambda x: preprocess_text(x))

In [11]:
tf_vectorizer = CountVectorizer(ngram_range = (1, 3),
                                max_df = 0.75,
                                min_df = 0.01,
                                tokenizer = tokenizer.tokenize,
                                stop_words='english'
)

In [12]:
df.head()

,title,link,citations,text,text_pp
0,Continual Reinforcement Learning with TELLA,http://arxiv.org/pdf/2208.04287v1,2,Workshop Track - 1st Conference on Lifelong Le...,workshop track 1st confer on lifelong learn ag...
1,An exact mapping between the Variational Renor...,http://arxiv.org/pdf/1410.3831v1,295,arXiv:1410.3831v1 [stat.ML] 14 Oct 2014An ex...,arxiv 1410 3831v1 stat ml 14 oct 2014an exact ...
2,Learning Generative Models across Incomparable...,http://arxiv.org/pdf/1905.05461v2,69,Learning Generative Models across Incomparable...,learn gener model across incompar space charlo...
3,On the Generalization Ability of Online Learni...,http://arxiv.org/pdf/1305.2505v1,74,On the Generalization Ability of Online Learni...,on the gener abil of onlin learn algorithm for...
4,Geometric Understanding of Deep Learning,http://arxiv.org/pdf/1805.10451v2,110,Geometric Understanding of Deep Learning\nNa L...,geometr understand of deep learn na lei zhongx...


In [13]:
tf = tf_vectorizer.fit_transform(df['text_pp'])

In [16]:
#params = {
#    'n_components': list(range(5, 150, 5)),
#    'learning_decay': np.arange(0.6, 0.9, 0.1),
#    'learning_offset': np.arange(10, 150, 10),
#    'perp_tol': [0.1],
#    'max_iter': [20],
#    'mean_change_tol': [1e-6, 1e-5, 1e-4, 1e-3, 1e-2],
#    'max_doc_update_iter': [100]
#}
best_params = {'perp_tol': 0.1,
 'n_components': 5,
 'mean_change_tol': 0.01,
 'max_iter': 20,
 'max_doc_update_iter': 100,
 'learning_offset': 30,
 'learning_decay': 0.6,
 'verbose': 3,
 'n_jobs': -1}

lda = LatentDirichletAllocation(**best_params)
#rsearch = RandomizedSearchCV(lda, params, n_iter=30, verbose=3, cv=3, error_score='raise')
#rsearch.fit(tf)
lda.fit(tf)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    6.8s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


iteration: 1 of max_iter: 20


[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    1.9s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


iteration: 2 of max_iter: 20


[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    1.6s finished
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


...

iteration: 19 of max_iter: 20


[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    1.2s finished


iteration: 20 of max_iter: 20


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    1.0s finished


LatentDirichletAllocation(learning_decay=0.6, learning_offset=30, max_iter=20,
                          mean_change_tol=0.01, n_components=5, n_jobs=-1,
                          verbose=3)

In [11]:
#best = rsearch.best_estimator_

In [17]:
#best.perplexity(tf)
lda.perplexity(tf)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    1.5s finished


13080.968063222543

In [18]:
pickle.dump(lda, open('model.pkl', 'wb'))

In [19]:
for index, component in enumerate(lda.components_): #taking model's components, i.e. P(w|z)
    zipped = zip(tf_vectorizer.get_feature_names(), component) #taking together tokens' names with components
    top_terms_key=sorted(zipped, key = lambda t: t[1], reverse=True)[:10] #top 10 terms per topic
    top_terms_list=list(dict(top_terms_key).keys()) #taking only tokens, no weights

    print("Topic "+str(index)+": ",top_terms_list) #prints top 10 tokens per topic

E:\anaconda\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Topic 0:  ['layer', 'dataset', '00', 'tabl', '100', 'nn', '0 0', 'class', 'accuraci', '000']
Topic 1:  ['z', 'bound', 'u', 'q', 'quantum', 'theorem', '1 2', 'probabl', 'xi', 'kernel']
Topic 2:  ['imag', 'pp', 'adversari', 'graph', 'attack', 'knowledg', 'represent', 'vol', 'structur', 'ieee']
Topic 3:  ['dataset', 'ml', 'http', 'user', '2018', '2019', 'differ', 'human', 'doi', 'decis']
Topic 4:  ['polici', 'ct', 'meta', 'agent', 'reinforc', 'reward', 'reinforc learn', 'action', 'd2', 'cp']


In [20]:
lda_output = lda.transform(tf)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    1.4s finished


In [21]:
topics = [f'Topic {i}' for i in range(lda.n_components)]
docnames = df.index.tolist()
topics_df = pd.DataFrame(lda_output, columns=topics, index=docnames)

In [22]:
topics_df['citations'] = df['citations']

In [23]:
xg_model = xgb.XGBRegressor()
xg_model.fit(topics_df[[f'Topic {i}' for i in range(lda.n_components)]], topics_df['citations'])

E:\anaconda\lib\site-packages\xgboost\data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [24]:
exp = dx.Explainer(xg_model, topics_df[['Topic 0', 'Topic 1', 'Topic 2', 'Topic 3', 'Topic 4']], topics_df['citations'])

Preparation of a new explainer is initiated

  -> data              : 1223 rows 5 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 1223 values
  -> model_class       : xgboost.sklearn.XGBRegressor (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_default at 0x0000028451EF19D0> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = -19.7, mean = 84.5, max = 1.16e+04
  -> model type        : regression will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -1.06e+02, mean = 0.104, max = 1.27e+02
  -> model_info        : package xgboost

A new explainer has been created!


In [30]:
agg_profiles = exp.model_profile().plot(show=False)
agg_profiles.write_html('agg_profiles.html')

Calculating ceteris paribus: 100%|██████████| 5/5 [00:00<00:00, 35.97it/s]


In [31]:
agg_profiles

In [32]:
var_importance = exp.model_parts().plot(show=False)
var_importance.write_html('var_importance.html')

In [33]:
var_importance

In [27]:
for index, component in enumerate(lda.components_): #taking model's components, i.e. P(w|z)
    zipped = zip(tf_vectorizer.get_feature_names(), component) #taking together tokens' names with components
    top_terms_key=sorted(zipped, key = lambda t: t[1], reverse=True)[:10] #top 10 terms per topic
    top_terms_list=list(dict(top_terms_key).keys()) #taking only tokens, no weights

    print("Topic "+str(index)+": ",top_terms_list) #prints top 10 tokens per topic

E:\anaconda\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning:

Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.



Topic 0:  ['layer', 'dataset', '00', 'tabl', '100', 'nn', '0 0', 'class', 'accuraci', '000']
Topic 1:  ['z', 'bound', 'u', 'q', 'quantum', 'theorem', '1 2', 'probabl', 'xi', 'kernel']
Topic 2:  ['imag', 'pp', 'adversari', 'graph', 'attack', 'knowledg', 'represent', 'vol', 'structur', 'ieee']
Topic 3:  ['dataset', 'ml', 'http', 'user', '2018', '2019', 'differ', 'human', 'doi', 'decis']
Topic 4:  ['polici', 'ct', 'meta', 'agent', 'reinforc', 'reward', 'reinforc learn', 'action', 'd2', 'cp']


In [36]:
exp.model_performance()

,mse,rmse,r2,mae,mad
XGBRegressor,399.014798,19.975355,0.997707,12.688974,8.000488
